In [1]:
import os 
import matplotlib.pyplot as plt
import re
import numpy as np

# objective of this file
## parse ref
## parse mel_pretrain
## somehow find the BPM from name and difficulty (have them cached perhaps?)
## calculate window for each entry
## see how 8th, 16th, 24th, 12th, 3rd, 2nd tokens have increased accuracy (downbeat excluded because ...)
### 4th = 24, 72
### triplets = 16, 32, 64, 80
### 8th = 12, 36, 60, 84
### 12th = 8, 40, 56, 88

In [2]:
def churry(lines):
    ret = []
    for line in lines:
        if '[INFO]' in line:
            ret.append(line.replace("\n",''))
        else:
            ret[-1] += f" {line.strip()}"
    return ret

def separate(lines):
    ret = []
    past = 0
    for n,line in enumerate(lines):
        if "precision" in line and "recall" in line and "f1" in line:
            ret.append(churry(lines[past:n+1]))
            past = n+1
    return ret


In [3]:
ref = "generate_ref.log"
subject = "mel_pretrain_epoch10.log" 
# the non-beat-aligned file was dubbed "pretrain", because initially we planned to start from this and finetune.
# That actually turned out to be worse and so we aborted, but the name stuck.

# subject = "mel_no_pretrain_epoch10.log" # uncomment this line to get stats for the beat-aligned one..

with open(subject) as file:
    sublines = file.readlines()

with open(ref) as file:
    reflines = file.readlines()

sub_sep = separate(sublines[2:])[:-1]
ref_sep = separate(reflines[2:])[:-1]

In [4]:
import re

fine_diff_crit = re.compile(r', ([0-9](\.[0-9]*)?)')
def finediff(lines):
    return float(fine_diff_crit.findall(lines[0])[-1][0])

def precision_recall_f1(lines):
    line = lines[-1]
    lines = line.split('[INFO] -')[-1].strip()
    l = lines.split('\t')
    ret = []
    for item, keyword in zip(l, ['precision','recall','f1']):
        ret.append(float(item.replace(keyword,'')))
    return ret

def length(lines):
    return 2*(len(lines)-2)

data_finediff_length_prf = [{
    "finediff": finediff(entry),
    "length": length(entry),
    "prf": precision_recall_f1(entry),
} for entry in sub_sep]

ref_finediff_length = [{
    "finediff": finediff(entry),
    "length": length(entry),
} for entry in ref_sep]


In [5]:
star_diffs = []
artists_and_titles = []

for ref_data, sub_data in zip(ref_sep, sub_sep):
    artist_title_star = ref_data[0].split("[INFO] - (")[-1][1:].replace(",)",'')[:-1]
    last_comma = artist_title_star.rfind(',')
    artist_title = artist_title_star[:last_comma].strip()
    star = artist_title_star[last_comma+1:].strip()
    star_diffs.append(float(star))
    artists_and_titles.append(artist_title)
    # double check with thing from sub
    crit = f" {star}',)"
    crit2 = f' {star}",)'
    if crit not in sub_data[0] and crit2 not in sub_data[0]:
        print(f"error: {artist_title_star}")
        raise ValueError
    



In [6]:
# load all osujsons
import os
import json
from pprint import pprint
folder = "(BIGFOLDER)" # <- replace this
split_json_path = f"{folder}/test.json"
bpms = []
with open(split_json_path) as file:
    split_dict = json.loads(file.read())
for n, (osujson_fname, (bar_len,beat_len)) in enumerate(split_dict.items()):
    with open(osujson_fname) as file: 
        osujson = json.load(file)
    songname = f'{osujson["artist"]} - {osujson["title"]}, {osujson["charts"][0]["difficulty_fine"]}'
    bpms.append(osujson['bpms'])
    



In [7]:
assert len(bpms) == len(ref_sep)
assert len(bpms) == len(sub_sep)

In [8]:
print(ref_sep[51][14])
print(sub_sep[51][14])

[2023-08-25 23:25:52,343][/mnt/c/Users/manym/Desktop/gorst/gorst/generate_ref.py][INFO] - 13		tensor([[  0, 132,  24,  97,  48,  99,  72, 123, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177, 177]], device='cuda:0')
[2023-08-23 08:42:22,092][/mnt/c/Users/manym/Desktop/gorst/gorst/metrics_cond_centered_AR.py][INFO] - 13		tensor([[ 23,  99,  47,  97,  71, 123,  95, 105]], device='cuda:0')


In [9]:
# 51 has crazy many ~es
#idx = 52

tensor_crit = re.compile(f"\[\[(.*)\]\]")

def classify_token(token):
    token = token % 48
    if token in [16,32]: return "3rds"
    elif token in [12,36]: return "4ths"
    elif token in [8,40]: return "6ths"
    elif token in [6,18,30,42]: return "8ths"
    elif token in [4*i for i in range(12)]: return "12ths"
    return None

def stats_for_token(sub_sep,ref_sep,bpms,token_to_find):
    true_pos, false_pos, false_neg=0,0,0
    for n,(line,refline) in enumerate(zip(sub_sep, ref_sep)):
        if n == 0: continue
        if n == len(sub_sep)-1: continue
        current_generated_beat = 2*n
        curbpm = bpms[0][1]
        for beat, bpm in bpms:
            if current_generated_beat >= beat:
                curbpm = bpm
            else: break
        line = tensor_crit.findall(line)
        try: line=line[0]
        except IndexError: line = ""
        refline = tensor_crit.findall(refline)
        try: refline=refline[0]
        except IndexError: refline="" 
        # window = 30ms. how many tokens is that??
        leniency = int ( 30 / (60000 / bpm / 48) )
        left_on_sup = max(token_to_find - leniency, 0)
        right_on_sup = min(token_to_find + leniency, 95)
        # 1. does the token exist in ref?
        ref_has = (f" {token_to_find}, " in refline)
        # 2. does the token exist in dat? token has to exist within the range
        dat_has = False
        for j in range(left_on_sup,right_on_sup+1):
            dat_has |= (f" {j}, " in line)
        if ref_has and dat_has:
            true_pos += 1
        elif ref_has and not dat_has:
            false_neg += 1
        elif not ref_has and dat_has:
            false_pos += 1
        #print('-------------------------')
        #print(refline.replace(" 177",'').strip(','))
        #print(line)
        #print(f"ref: {ref_has}, dat: {dat_has}")
    return true_pos, false_neg, false_pos

toc = {
    "8ths":[24,72],
    "12ths":[16,32,64,80],
    "16ths":[12,36,60,84],
    "24ths":[8,40,56,88],
    "32nds":[6,18,30,42,54,66,78,90],
    "48ths":[4,20,28,44,52,68,76,92]
}

for key in toc.keys():
    true_pos, false_neg, false_pos = 0,0,0
    for idx in range(len(bpms)):
        for token in toc[key]:
            o = stats_for_token(sub_sep[idx], ref_sep[idx], bpms[idx], token)
            true_pos += o[0]
            false_neg += o[1]
            false_pos += o[2]
        #print(f"rp {true_pos}; fn {false_neg}; fp {false_pos}")
    print('================')
    print(key)
    print(f"tp {true_pos}; fn {false_neg}; fp {false_pos}")
    # therefore, micro-f1 precision and recall would be:
    print(f"precision {true_pos / (true_pos + false_pos)}")
    print(f"recall {true_pos / (true_pos + false_neg)}")
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    print(f"f1 {2*precision * recall / (precision + recall)}")

    
    

8ths
tp 109725; fn 59240; fp 34367
precision 0.7614926574688393
recall 0.6493948450862604
f1 0.7009905544357736
12ths
tp 5911; fn 10842; fp 106353
precision 0.05265267583553054
recall 0.35283232853817226
f1 0.09163133540541168
16ths
tp 59016; fn 52937; fp 77047
precision 0.4337402526770687
recall 0.5271497860709404
f1 0.47590478033675243
24ths
tp 2420; fn 4397; fp 113754
precision 0.02083082273142011
recall 0.3549948657767346
f1 0.03935247294517485
32nds
tp 2843; fn 6845; fp 161276
precision 0.017322796263686716
recall 0.2934558216350124
f1 0.03271444763444511
48ths
tp 1113; fn 1729; fp 415023
precision 0.002674606378683892
recall 0.3916256157635468
f1 0.005312928125104421


In [10]:
# now, let us do the same for the "anmillin":
ref = "generate_ref.log"
# subject = "mel_pretrain_epoch10.log"
subject = "mel_no_pretrain_epoch10.log"

with open(subject) as file:
    sublines = file.readlines()

with open(ref) as file:
    reflines = file.readlines()

sub_sep = separate(sublines[2:])[:-1]
ref_sep = separate(reflines[2:])[:-1]

In [12]:
tensor_crit = re.compile(f"\[\[(.*)\]\]")

def classify_token(token):
    token = token % 48
    if token in [16,32]: return "3rds"
    elif token in [12,36]: return "4ths"
    elif token in [8,40]: return "6ths"
    elif token in [6,18,30,42]: return "8ths"
    elif token in [4*i for i in range(12)]: return "12ths"
    return None

def stats_for_token(sub_sep,ref_sep,token_to_find):
    true_pos, false_pos, false_neg=0,0,0
    for n,(line,refline) in enumerate(zip(sub_sep, ref_sep)):
        line = tensor_crit.findall(line)
        try: line=line[0]
        except IndexError: line = ""
        refline = tensor_crit.findall(refline)
        try: refline=refline[0]
        except IndexError: refline="" 
        # 1. does the token exist in ref?
        ref_has = (f" {token_to_find}, " in refline)
        # 2. does the token exist in dat? 
        dat_has = (f" {token_to_find}, " in line)
        if ref_has and dat_has:
            true_pos += 1
        elif ref_has and not dat_has:
            false_neg += 1
        elif not ref_has and dat_has:
            false_pos += 1
        #print('-------------------------')
        #print(refline.replace(" 177",'').strip(','))
        #print(line)
        #print(f"ref: {ref_has}, dat: {dat_has}")
    return true_pos, false_neg, false_pos

toc = {
    "8ths":[24,72],
    "12ths":[16,32,64,80],
    "16ths":[12,36,60,84],
    "24ths":[8,40,56,88],
    "32nds":[6,18,30,42,54,66,78,90],
    "48ths":[4,20,28,44,52,68,76,92]
}

for key in toc.keys():
    true_pos, false_neg, false_pos = 0,0,0
    for idx in range(len(bpms)):
        for token in toc[key]:
            o = stats_for_token(sub_sep[idx], ref_sep[idx], token)
            true_pos += o[0]
            false_neg += o[1]
            false_pos += o[2]
        #print(f"rp {true_pos}; fn {false_neg}; fp {false_pos}")
    print('================')
    print(key)
    print(f"tp {true_pos}; fn {false_neg}; fp {false_pos}")
    # therefore, micro-f1 precision and recall would be:
    print(f"precision {true_pos / (true_pos + false_pos)}")
    print(f"recall {true_pos / (true_pos + false_neg)}")
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    print(f"f1 {2*precision * recall / (precision + recall)}")

8ths
tp 149777; fn 19188; fp 22127
precision 0.8712828090096798
recall 0.8864380197082236
f1 0.8787950796346984
12ths
tp 8629; fn 8124; fp 3777
precision 0.6955505400612607
recall 0.5150719274159852
f1 0.5918584313591001
16ths
tp 61562; fn 50391; fp 18978
precision 0.7643655326545816
recall 0.5498914723142747
f1 0.6396284540217047
24ths
tp 1204; fn 5613; fp 1145
precision 0.5125585355470413
recall 0.17661728032859028
f1 0.2627100152738381
32nds
tp 1471; fn 8217; fp 2543
precision 0.36646736422521176
recall 0.1518373245251858
f1 0.21471318055758284
48ths
tp 22; fn 2820; fp 105
precision 0.1732283464566929
recall 0.007741027445460943
f1 0.01481980464802964


In [13]:
# from ref, how many tokens are 8/12/16/24th?
toc = {
    "4ths":[0,48],
    "8ths":[24,72],
    "12ths":[16,32,64,80],
    "16ths":[12,36,60,84],
    "24ths":[8,40,56,88],
    "32nds":[6,18,30,42,54,66,78,90],
    "48ths":[4,20,28,44,52,68,76,92]
}

tally = [0 for _ in range(96)]

for ref_sep_entry in ref_sep:
    for refline in ref_sep_entry:
        refline = tensor_crit.findall(refline)
        try: refline=refline[0]
        except IndexError: continue
        # 1. does the token exist in ref?
        refline = [int(e.strip()) for e in refline.split(',')]
        for token in refline:
            if token < 96: tally[token] += 1

# for n,t in enumerate(tally):
#    print(n,t)
totl = sum(tally)
for category in toc:
    counts = sum([tally[token] for token in toc[category]])
    print(f"{category}\t{counts/totl}")

4ths	0.4059714984874896
8ths	0.3129961043363571
12ths	0.031033786499849027
16ths	0.20738527428028403
24ths	0.012628026178563291
32nds	0.017946357285891324
48ths	0.0052646105910923975
